In [2]:
import pandas as pd
import seaborn as sns
import gc
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import statsmodels.api as sm
import scipy
import plotly.express as px
from scipy.fft import fft
from scipy.signal import lombscargle
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from pprint import pprint
from scipy import stats
import random
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from scipy.stats import ttest_ind
from matplotlib.pyplot import tick_params
from sklearn.impute import SimpleImputer

pl.disable_string_cache()
pl.Config.set_streaming_chunk_size(10000)

polars.config.Config

In [3]:
columns = [ "Epoch", "range_km", "Mag", "sat_j2000", "obs_j2000", "az_rad", "el_rad", 'phase_angle_rad', 'Channel','Filter', "Track", "epsecs", "Satellite"]

N = 300000000
lf = pl.scan_parquet("mmt.parquet").limit(n=N).select(columns)
print(lf.collect_schema())

Schema({'Epoch': Datetime(time_unit='ms', time_zone='UTC'), 'range_km': Float32, 'Mag': Float32, 'sat_j2000': Array(Float32, shape=(3,)), 'obs_j2000': Array(Float32, shape=(3,)), 'az_rad': Float32, 'el_rad': Float32, 'phase_angle_rad': Float32, 'Channel': UInt8, 'Filter': String, 'Track': UInt32, 'epsecs': Float32, 'Satellite': UInt32})


In [4]:
# Sampling the dataset
sample_rate = 0.01
lf = lf.with_row_index("row_num")

sampled_lf = lf.filter(pl.col("row_num") % int(1/sample_rate) == 0)
sampled_df = sampled_lf.collect()

print(sampled_df.shape)
sampled_df.describe()

(2786335, 14)


statistic,row_num,Epoch,range_km,Mag,sat_j2000,obs_j2000,az_rad,el_rad,phase_angle_rad,Channel,Filter,Track,epsecs,Satellite
str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64
"""count""",2.786335e6,"""2786335""",2.786335e6,2.786335e6,2.786335e6,2.786335e6,2.786335e6,2.786335e6,2.786335e6,2.786335e6,"""2786335""",2.786335e6,2.786335e6,2.786335e6
"""null_count""",0.0,"""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""0""",0.0,0.0,0.0
"""mean""",1.393167e8,"""2020-04-06 23:04:26.625000+00:…",7644.399414,7.562555,null,null,3.166827,0.956153,1.087673,4.949778,null,1.7603e7,384.484344,32638.117467
"""std""",8.0435e7,null,9530.039062,1.462786,null,null,1.589515,0.246466,0.458938,2.492863,null,6.6430e6,1105.687744,14580.347077
"""min""",0.0,"""2014-06-04 19:53:39.246000+00:…",289.368011,-2.14178,null,null,0.000003,0.000079,0.000901,1.0,"""B""",10.0,0.0,5.0
"""25%""",6.96584e7,"""2018-01-06 23:05:43.605000+00:…",1219.544556,6.6205,null,null,2.177478,0.779899,0.730134,3.0,null,1.2913422e7,24.501001,25619.0
"""50%""",1.393167e8,"""2020-05-20 18:27:38.596000+00:…",2258.177002,7.79982,null,null,3.127978,0.955179,1.103191,5.0,null,1.7560417e7,67.401001,36869.0
"""75%""",2.089751e8,"""2022-11-13 15:49:33.396000+00:…",11763.980469,8.6941,null,null,4.243509,1.125711,1.440736,7.0,null,2.1884048e7,297.600006,43583.0
"""max""",2.786334e8,"""2024-12-01 03:31:54.098000+00:…",45000.234375,11.5391,null,null,6.283182,1.570057,2.841413,9.0,"""V""",3.0228937e7,38355.511719,87692.0


In [5]:
sampled_df = sampled_df.drop_nans()
sampled_df = sampled_df.drop_nulls()
sampled_df.null_count()

row_num,Epoch,range_km,Mag,sat_j2000,obs_j2000,az_rad,el_rad,phase_angle_rad,Channel,Filter,Track,epsecs,Satellite
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
satellite_counts = sampled_df['Satellite'].value_counts()

sorted_satellite_counts = satellite_counts.sort('count', descending=True)

print(sorted_satellite_counts.head(20))
print(sorted_satellite_counts.tail(10))

shape: (20, 2)
┌───────────┬───────┐
│ Satellite ┆ count │
│ ---       ┆ ---   │
│ u32       ┆ u32   │
╞═══════════╪═══════╡
│ 40750     ┆ 23722 │
│ 29517     ┆ 20517 │
│ 43247     ┆ 16812 │
│ 37818     ┆ 16789 │
│ 43624     ┆ 14708 │
│ …         ┆ …     │
│ 37385     ┆ 10816 │
│ 44034     ┆ 10763 │
│ 43612     ┆ 10702 │
│ 26383     ┆ 10551 │
│ 43209     ┆ 10545 │
└───────────┴───────┘
shape: (10, 2)
┌───────────┬───────┐
│ Satellite ┆ count │
│ ---       ┆ ---   │
│ u32       ┆ u32   │
╞═══════════╪═══════╡
│ 49395     ┆ 1     │
│ 5         ┆ 1     │
│ 44409     ┆ 1     │
│ 39134     ┆ 1     │
│ 59894     ┆ 1     │
│ 40723     ┆ 1     │
│ 59901     ┆ 1     │
│ 59649     ┆ 1     │
│ 59137     ┆ 1     │
│ 41776     ┆ 1     │
└───────────┴───────┘


### Observations
# The most observed satellites are: 
│ 40750     ┆ 23722 │ which is CZ-3B R/B \
│ 29517     ┆ 20517 │ which is CZ-3 R/B \
│ 43247     ┆ 16812 │ which is CZ-4B R/B 


In [15]:
from scipy.stats import entropy
from concurrent.futures import ThreadPoolExecutor


def spectral_entropy(signal):
    power_spectrum = np.abs(fft(signal - np.mean(signal))) ** 2
    norm_power = power_spectrum / power_spectrum.sum()
    return entropy(norm_power)

def lombscargle_power(times, magnitudes):
    frequencies = np.linspace(0.01, 10, 1000)
    power = lombscargle(times, magnitudes, frequencies)
    return power

def detect_anomalies(sat_id, min_obs=50):
    sat_data = sampled_df.filter(pl.col("Satellite") == sat_id)
    
    # Skip satellites with insufficient data
    if len(sat_data) < min_obs:
        return None

    features = {
        "satellite_id": sat_id,
        "obs_count": len(sat_data),
        "std_dev": sat_data["Mag"].std(),
        "max_power": lombscargle_power(sat_data["epsecs"], sat_data["Mag"]).max(),
        "entropy": spectral_entropy(sat_data["Mag"]),
        "track_count": sat_data["Track"].n_unique()
    }
    return features

anomaly_data = (
    sampled_df.lazy()
    .group_by("Satellite")
    .agg(pl.all())
    .collect()
    .to_dicts()
)

with ThreadPoolExecutor() as executor:
    anomaly_scores = list(executor.map(
        lambda x: detect_anomalies(x['Satellite']),
        anomaly_data
    ))

# Removing None values from satellites with insufficient data
anomaly_scores = [x for x in anomaly_scores if x is not None]

[{'std_dev': None, 'max_power': np.float64(40.11777508015514), 'entropy': np.float32(0.0)}, {'std_dev': 0.7221033573150635, 'max_power': np.float64(556.2971870812269), 'entropy': np.float32(0.048507467)}, {'std_dev': 0.6161327362060547, 'max_power': np.float64(16770.80184697001), 'entropy': np.float32(0.0483224)}, {'std_dev': 0.5119982361793518, 'max_power': np.float64(4686.646469329671), 'entropy': np.float32(0.030619914)}, {'std_dev': 0.3794952630996704, 'max_power': np.float64(1672.5754266813447), 'entropy': np.float32(0.017100781)}, {'std_dev': 0.6460438966751099, 'max_power': np.float64(11151.89662958175), 'entropy': np.float32(0.076516)}, {'std_dev': 0.8308889865875244, 'max_power': np.float64(17201.279222783032), 'entropy': np.float32(0.13308336)}, {'std_dev': 0.6029897928237915, 'max_power': np.float64(15606.134081947674), 'entropy': np.float32(0.08309468)}, {'std_dev': 0.5658321976661682, 'max_power': np.float64(2752.771194168915), 'entropy': np.float32(0.046836354)}, {'std_de

In [22]:
from sklearn.ensemble import IsolationForest
anomaly_df = pl.DataFrame(anomaly_scores)

features = ['std_dev', 'max_power', 'entropy']
scaled_features = anomaly_df.with_columns([
    ((pl.col(f) - pl.col(f).mean()) / pl.col(f).std()).alias(f) for f in features
])

clf = IsolationForest(contamination=0.01)
anomalies = clf.fit_predict(scaled_features.select(features).to_numpy())
anomaly_df = anomaly_df.with_columns(pl.lit(anomalies).alias('anomaly_score'))

top_anomalies = anomaly_df.filter(pl.col('anomaly_score') == -1).sort(
    by=['std_dev', 'max_power'],
    descending=True
)
print("anomaly_df shape: ", anomaly_df.shape)
print(top_anomalies)
print("top anomaly shape: ",top_anomalies.shape)

anomaly_df shape:  (12130, 4)
shape: (122, 4)
┌──────────┬───────────────┬──────────┬───────────────┐
│ std_dev  ┆ max_power     ┆ entropy  ┆ anomaly_score │
│ ---      ┆ ---           ┆ ---      ┆ ---           │
│ f64      ┆ f64           ┆ f64      ┆ i64           │
╞══════════╪═══════════════╪══════════╪═══════════════╡
│ 3.298859 ┆ 23.883488     ┆ 1.372085 ┆ -1            │
│ 2.511993 ┆ 88.783226     ┆ 1.576353 ┆ -1            │
│ 2.338051 ┆ 250.165303    ┆ 0.914037 ┆ -1            │
│ 2.324163 ┆ 543.601563    ┆ 1.186868 ┆ -1            │
│ 2.282099 ┆ 76.142576     ┆ 1.14982  ┆ -1            │
│ …        ┆ …             ┆ …        ┆ …             │
│ 0.621296 ┆ 101808.678989 ┆ 0.093825 ┆ -1            │
│ 0.615992 ┆ 95090.395339  ┆ 0.090865 ┆ -1            │
│ 0.603011 ┆ 95039.736505  ┆ 0.088644 ┆ -1            │
│ 0.355928 ┆ 67928.629335  ┆ 0.021416 ┆ -1            │
│ 0.318221 ┆ 110325.261894 ┆ 0.018026 ┆ -1            │
└──────────┴───────────────┴──────────┴───────────────┘
to

# Anomaly Detection Report

## Key Metrics
- Analyzed 12130 satellites
- Flagged 122 anomalies (1.005% of population)

## Characteristics of Anomalies
| Metric          | Anomalous Range       | Typical Range       |
|-----------------|-----------------------|---------------------|
| Brightness σ    | > 2.5 mag             | < 1.0 mag           |
| Max Power       | > 150 (arb. units)    | < 50 (arb. units)   |
| Spectral Entropy| > 6.5 bits            | < 5.0 bits          |

The anomalies will manifest as:
High temporal variability (large σ)
Strong periodic signals (high max_power)
Unpredictable patterns (high entropy)
Combinations of these features